In [17]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

from langchain_classic.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

In [18]:
# Carregando variaveis de ambiente

from dotenv import load_dotenv
load_dotenv()

True

In [19]:
# Define p array de perguntas
perguntas = [
    "Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?",
    "Quais são as principais características da população sertaneja descritas por Euclides da Cunha? Como ele relaciona essas características com o ambiente em que vivem?",
    "Qual foi o contexto histórico e político que levou à Guerra de Canudos, segundo Euclides da Cunha?",
    "Como Euclides da Cunha descreve a figura de Antônio Conselheiro e seu papel na Guerra de Canudos?",
    "Quais são os principais aspectos da crítica social e política presentes em 'Os Sertões'? Como esses aspectos refletem a visão do autor sobre o Brasil da época?"
]

In [20]:
# Carregar  modelos OpenAI - Embeddings e chat

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
llm = ChatOpenAI(
    model_name = "gpt-3.5-turbo",
    max_tokens = 500
)

In [21]:
# Carregar o PDF

pdf_link = "../os-sertoes.pdf"

loader = PyPDFLoader(pdf_link, extract_images=False)

pages = loader.load_and_split()

In [22]:
# separar em chunks

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 4000,
    chunk_overlap = 200,
    length_function = len,
    add_start_index=True
)

chunks = text_splitter.split_documents(pages)

In [23]:
# salvar os chunks no VectorDB

vectordb = Chroma.from_documents(chunks, embedding=embeddings, persist_directory="naiveDB")

#carrega o DB

naive_retriever = vectordb.as_retriever(search_kwargs={"k": 10})

In [24]:
# rerank

rerank =  CohereRerank(model="rerank-v3.5",top_n=3)

compressor_retriver = ContextualCompressionRetriever(
    base_compressor=rerank,
    base_retriever= naive_retriever
)

In [25]:
#PROMPT

TEMPLATE = """
    Você é um experiente analisador de obras literárias. Responda a pergunta abaixo utilizando o contexto informado.
    Query:
    {question}
    Context:
    {context}
"""

rag_prompt = ChatPromptTemplate.from_template(TEMPLATE);

In [27]:
setup_retrivel = RunnableParallel({"question": RunnablePassthrough(), "context": compressor_retriver})
output_parser = StrOutputParser()

compressor_retriver_chain = setup_retrivel | rag_prompt | llm | output_parser

for p in perguntas:
    resposta = compressor_retriver_chain.invoke(p)
    print(f"Pergunta:\n{p}\n")
    print(f"Resposta:\n{resposta}")
    print("-" * 80)

Pergunta:
Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?

Resposta:
Euclides da Cunha retrata o ambiente natural do sertão nordestino como um vale fértil, um pomar vastíssimo sem dono. No entanto, ele também descreve a atmosfera asfixiadora, o empedramento do solo, a nudez da flora e a seca assombradora que caracterizam a região. Ele destaca a natureza como um jogo de antíteses, com divisões entre os vales férteis e as estepes áridas. Essas condições influenciam a vida dos habitantes do sertão, sendo um desafio constante enfrentar os dias torturantes e a seca que assola a região. Euclides da Cunha enfatiza que a natureza não cria desertos, mas combate-os, e que as condições extremas do sertão nordestino impõem uma divisão especial no cenário, influenciando a economia da terra e a vida dos habitantes locais.
--------------------------------------------------------------------------------
Pergunta:
Quais são 